# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 0
inference = not finetuning

# Drive Mount

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# Read Data

In [2]:
import pandas as pd
# old_data = pd.read_csv('/kaggle/input/gb-all-necessary/v2(gender_code_change)_updated_SarcaSm.csv')
ndata = pd.read_csv('train_test_sarcasm_data_mod.csv')

In [3]:
ndata

,male,female,_original_label,test
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0
...,...,...,...,...
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1
9247,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,0,0
9248,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাট সাক...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাজ্ঞী ...,0,0


# Gender Masking

In [4]:
mf = pd.read_csv('male_female.csv')
male_gender_term_list = mf['Masculine'].tolist()
male_gender_term_set= set(male_gender_term_list).union({'ভাই'})
female_gender_term_list = mf['Feminine'].tolist()
female_gender_term_set = set(female_gender_term_list)

import re
from tqdm import tqdm
tqdm.pandas()

def mask_gender(text, female_gender_term_set, male_gender_term_set):

    mod_sen = []
    for word in text.split():
      if word in male_gender_term_set.union(female_gender_term_set):
        mod_sen.append('<Gender>')
        continue
      mod_sen.append(word)
    return " ".join(mod_sen)

ndata['gm_male'] = ndata['male'].progress_apply(lambda x: mask_gender(x, female_gender_term_set, male_gender_term_set))
ndata['gm_female'] = ndata['female'].progress_apply(lambda x: mask_gender(x, female_gender_term_set, male_gender_term_set))
ndata

100%|██████████| 9250/9250 [00:01<00:00, 9190.53it/s]


,male,female,_original_label,test,gm_male,gm_female
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুতি...,নিশ্চয় <Gender> সাংবাদিককে কোনো কিছু দেওয়ার প্...
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...
...,...,...,...,...,...,...
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...
9247,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,0,0,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...
9248,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাট সাক...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাজ্ঞী ...,0,0,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার <Gender> স...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার <Gender> স...


# Name Mask

In [5]:
i = 0
gn_masked = []
for i in range(len(ndata)):
  sd = set(ndata.gm_female.iloc[i].split(' ')) -  set(ndata.gm_male.iloc[i].split(' '))
  # print(ndata.gm_female.iloc[i])
  # print(ndata.gm_male.iloc[i])
  st = ndata.gm_female.iloc[i]
  for x in sd:
    st = st.replace(x,'<Name>')
  gn_masked.append(st)
  # if i == 50:
  #   break
  # print(ndata.gm_female.iloc[i])
  # print(st)
ndata['Gender and Name Masked'] = gn_masked
ndata

,male,female,_original_label,test,gm_male,gm_female,Gender and Name Masked
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুতি...,নিশ্চয় <Gender> সাংবাদিককে কোনো কিছু দেওয়ার প্...,নিশ্চয় <Gender> সাংবাদিককে কোনো কিছু দেওয়ার প্...
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...
...,...,...,...,...,...,...,...
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...
9247,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,0,0,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...
9248,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাট সাক...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাজ্ঞী ...,0,0,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার <Gender> স...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার <Gender> স...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার <Gender> <...


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [12]:
temp = ndata[ndata['test']!= 1]
test = ndata[ndata['test']== 1]
# validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=1234))
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.02, random_state=1234))
train = temp.drop(validation.index)
print('Train class distribution: ', train._original_label.value_counts())
diff  = max(train._original_label.value_counts()) - min(train._original_label.value_counts())
print('Difference (Change carefully): ', diff)
ex = train[train._original_label==0].sample(n = diff, random_state = 1234)
train = pd.concat([train, ex])
train = train.reset_index(drop=True)
print('Train class distribution: ', train._original_label.value_counts())

Train class distribution:  _original_label
1    4112
0    3140
Name: count, dtype: int64
Difference (Change carefully):  972
Train class distribution:  _original_label
1    4112
0    4112
Name: count, dtype: int64


C:\Users\austg\AppData\Local\Temp\ipykernel_16408\2940535021.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.02, random_state=1234))


# Gender and Name Masking With Finetuning

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
    
model_path = "raquiba/sarcasm-detection-BanglaSARC"


tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at raquiba/sarcasm-detection-BanglaSARC and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Pretrained tokens:  32000
Total number of tokens after adding the new ones:  32002


In [14]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length = 512):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,2]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,6] #6th column Gender and Name Masked

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer )
dataset_val= BertDataset( validation, tokenizer
                        )
dataset_test= BertDataset(test, tokenizer)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle=False)

In [14]:
# id,am, tid, l = next(iter(train_loader))
# x = b_model(id, am,tid)
# x.hidden_states[0].shape

In [15]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)

In [16]:
# id,am, tid, l = next(iter(train_loader))
# x,_ = masked_model(id, am,tid)
# x.shape

In [16]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


In [18]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = '2,3.Sarcasm_Masked.pth')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'2,3.Sarcasm_Masked.pth' 
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return

# finetuning = 1
if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.042718,0.554353,0.044806,0.675676,51.840144,51.842143,0.0001,16,cuda
1,1,2,0.039036,0.664154,0.040344,0.770270,51.370290,104.411918,0.0001,16,cuda
2,1,3,0.034133,0.772860,0.038259,0.722973,51.278368,156.878506,0.0001,16,cuda
3,1,4,0.028794,0.836819,0.032584,0.810811,51.289592,209.343406,0.0001,16,cuda
4,1,5,0.024320,0.866245,0.029971,0.824324,52.039859,262.507730,0.0001,16,cuda
5,1,6,0.020584,0.893361,0.027620,0.824324,51.210589,314.878320,0.0001,16,cuda
6,1,7,0.017427,0.910992,0.025930,0.837838,52.035749,368.053359,0.0001,16,cuda
7,1,8,0.014886,0.930569,0.024537,0.837838,51.265805,420.437796,0.0001,16,cuda
8,1,9,0.013272,0.938108,0.023293,0.831081,51.748876,473.336946,0.0001,16,cuda
9,1,10,0.011694,0.949052,0.022701,0.837838,51.790585,526.268421,0.0001,16,cuda


Saved model and optimizer at 2,3.Sarcasm_Masked.pth


In [19]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
#   load_model(masked_model, None, PATH = '/kaggle/input/sarcasm_masked/pytorch/default/1/23.Sarcasm_Masked.pth')
  load_model(masked_model, None, PATH = '2,3.Sarcasm_Masked.pth')

Loaded model from 2,3.Sarcasm_Masked.pth


In [21]:
def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()
dataset_test= BertDataset(test, tokenizer)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['token_masked_pred'] = [x[0] for pred in p for x in pred.tolist()]

  0%|          | 0/116 [00:00<?, ?it/s]c:\Users\austg\.conda\envs\Temp\Lib\site-packages\transformers\tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 116/116 [00:08<00:00, 13.47it/s]
C:\Users\austg\AppData\Local\Temp\ipykernel_16408\3411750198.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['token_mas

In [22]:
test

,male,female,_original_label,test,gm_male,gm_female,Gender and Name Masked,token_masked_pred
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,1,1,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,<Gender> মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ ক...,1
11,দে বাবা দে বলি দে । জবা গলা জোড়াতে ও পারে,দে মা দে বলি দে । জবা গলা জোড়াতে ও পারে,1,1,দে <Gender> দে বলি দে । জবা গলা জোড়াতে ও পারে,দে <Gender> দে বলি দে । জবা গলা জোড়াতে ও পারে,দে <Gender> দে বলি দে । জবা গলা জোড়াতে ও পারে,1
20,আপনি কি জন্য আপনার বাপের বয়সী বেডার লগে ক্লাবে...,আপনি কি জন্য আপনার মায়ের বয়সী বেডির লগে ক্লাবে...,1,1,আপনি কি জন্য আপনার <Gender> বয়সী বেডার লগে ক্ল...,আপনি কি জন্য আপনার <Gender> বয়সী বেডির লগে ক্ল...,আপনি কি জন্য আপনার <Gender> বয়সী <Name> লগে ক্...,0
29,বন্ধু মানুষ আসলে বাঙ্গির মতো ।,বন্ধুপত্নী মানুষ আসলে বাঙ্গির মতো ।,1,1,<Gender> মানুষ আসলে বাঙ্গির মতো ।,<Gender> মানুষ আসলে বাঙ্গির মতো ।,<Gender> মানুষ আসলে বাঙ্গির মতো ।,1
34,আয় হায় ! কি ঢস টা দিলো রে আতিক ভাই !,আয় হায় ! কি ঢস টা দিলো রে বিনিতা বোন !,1,1,আয় হায় ! কি ঢস টা দিলো রে আতিক <Gender> !,আয় হায় ! কি ঢস টা দিলো রে বিনিতা <Gender> !,আয় হায় ! কি ঢস টা দিলো রে <Name> <Gender> !,1
...,...,...,...,...,...,...,...,...
9242,সমানভাবে ভাগ করায় সাংবাদিক সাহেবও খুশি,সমানভাবে ভাগ করায় মহিলা সাংবাদিক সাহেবও খুশি,0,1,সমানভাবে ভাগ করায় <Gender> সাহেবও খুশি,সমানভাবে ভাগ করায় <Gender> <Gender> সাহেবও খুশি,সমানভাবে ভাগ করায় <Gender> <Gender> সাহেবও খুশি,0
9243,ত্রিশ দিন রমজানের পর আজকে শয়তান ছাড়া পেয়েছে ৷,ত্রিশ দিন রমজানের পর আজকে শয়তানি ছাড়া পেয়েছে ৷,0,1,ত্রিশ দিন রমজানের পর আজকে শয়তান ছাড়া পেয়েছে ৷,ত্রিশ দিন রমজানের পর আজকে শয়তানি ছাড়া পেয়েছে ৷,ত্রিশ দিন রমজানের পর আজকে <Name> ছাড়া পেয়েছে ৷,1
9245,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,0,1,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,1
9246,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,0,1,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,ডিজিটাল মিটার চালু হইছে তো <Gender> অনেক আগে থ...,1


# Token Masked Result

In [23]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []
    
    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])
    
    return original_acc, (lower, upper), np.mean(bootstrap_accs)


y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['token_masked_pred'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
test.to_csv('2,3.revision_tm_sarcasm_result.csv', index = False)

Accuracy: 0.8708
95% Confidence Interval: (np.float64(0.8545945945945946), np.float64(0.885945945945946))
mean accuracy: 0.8706745945945945


# Sentence Embedding Visualization for Masked Finetuned Model

In [62]:
# #for visualizing training data embedding
# dataset_train= BertDataset(train, tokenizer, max_length=200)
# train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
# p, emb = get_prediction(train_loader, masked_model)

In [63]:
# from sklearn.manifold import TSNE
# import plotly.express as px

# tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb)

# labels = train['original_lebel']   # Extract labels
# review_texts = train['Gender and Name Masked']  # Extract review texts

# # Prepare the data for Plotly plots
# plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
#                  't-SNE Dimension 2': tsne_cls[:, 1],
#                  'Label': labels,
#                  'Review Text': review_texts}

# # Customize colorbar to show integer labels
# colorbar_tickvals = list(range(1, 6))
# colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# # Plotting with Plotly - [CLS] Token Embeddings
# fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
#                      color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Review Text': True})


# fig_cls.update_layout(title='t-SNE of Embeddings (Finetuned on GN Masked Token) with Labels', height=700,  font=dict(
#         size=20,
# ))
# fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

# fig_cls.update_layout(coloraxis_colorbar=dict(
#     tickvals=colorbar_tickvals,
#     ticktext=colorbar_ticktext
# ))

# fig_cls.show()


# Finetuning with original data

In [25]:
# #data preparation
# temp = ndata[ndata['test']!= 1]
# test = ndata[ndata['test']== 1]
# validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=2111))
# train = temp.drop(validation.index)

import numpy as np
np.random.seed(699)
def n_train(x):
    c = np.random.randint(0, 2)
    if c == 0:
        input_str = x[0]
    else:
        input_str = x[1]
    return input_str
        
train['original_data'] = train.apply(lambda x: n_train(x), axis = 1)
train = train[['original_data', 'female', '_original_label', 'test', 'male']]
train.head(15)

C:\Users\austg\AppData\Local\Temp\ipykernel_16408\4084897414.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input_str = x[1]
C:\Users\austg\AppData\Local\Temp\ipykernel_16408\4084897414.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input_str = x[0]


,original_data,female,_original_label,test,male
0,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...
1,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,1,0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...
2,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,0,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...
3,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,1,0,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...
5,ওদের কথায় বুঝা যায় সরকার খাইতে দিছে এখন ঘুমানো...,ওদের কথায় বুঝা যায় সরকার খাইতে দিছে এখন ঘুমানো...,1,0,ওদের কথায় বুঝা যায় সরকার খাইতে দিছে এখন ঘুমানো...
6,এই হারামজাদী গুলো প্রভাব খাটাতে পারে শুধুমাত্র...,এই হারামজাদী গুলো প্রভাব খাটাতে পারে শুধুমাত্র...,1,0,এই হারামজাদা গুলো প্রভাব খাটাতে পারে শুধুমাত্র...
7,আমার এত সময় নাই । অন্তরা বিবি কচু ক্ষেতের ...,আমার এত সময় নাই । অন্তরা বিবি কচু ক্ষেতের ...,1,0,আমার এত সময় নাই । মতি মিয়া কচু ক্ষেতের ধার ধারেনা
8,অনেকদিন দিন পর কাকু কে দেখলাম ।,অনেকদিন দিন পর কাকী কে দেখলাম ।,1,0,অনেকদিন দিন পর কাকু কে দেখলাম ।
9,গোলাপ ফুলের জায়গায় আমি দিলাম তোমায় জবা বলো তুম...,গোলাপ ফুলের জায়গায় আমি দিলাম তোমায় জবা বলো তুম...,1,0,গোলাপ ফুলের জায়গায় আমি দিলাম তোমায় জবা বলো তুম...


In [28]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length = 512):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,2]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th coumn meanS original

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer )
dataset_val= BertDataset( validation, tokenizer)
dataset_test= BertDataset(test, tokenizer)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle=False)
ori_model=BERT(b_model)

In [29]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = '2,3.Sarcasm_Original.pth')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'2,3.Sarcasm_Original.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return

# inference = 1
# finetuning = 1
if finetuning:
  my_trainer(train_loader, validation_loader, ori_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)
else:
    if inference:
      load_model(ori_model, None, PATH = '2,3.Sarcasm_Original.pth')

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.033840,0.757053,0.031128,0.858108,51.896111,51.898114,0.0001,16,cuda
1,1,2,0.019732,0.919869,0.025377,0.851351,51.341176,104.375075,0.0001,16,cuda
2,1,3,0.013540,0.938351,0.022283,0.858108,51.280289,156.799908,0.0001,16,cuda
3,1,4,0.010059,0.956104,0.021028,0.871622,51.971239,209.916881,0.0001,16,cuda
4,1,5,0.007712,0.964859,0.020379,0.871622,51.137283,262.162086,0.0001,16,cuda
5,1,6,0.006347,0.971425,0.019895,0.871622,51.945540,315.235677,0.0001,16,cuda
6,1,7,0.005162,0.978113,0.019905,0.878378,51.266716,367.624573,0.0001,16,cuda
7,1,8,0.004246,0.984193,0.020020,0.871622,51.345467,420.112254,0.0001,16,cuda
8,1,9,0.003560,0.987719,0.020337,0.885135,51.540184,472.789408,0.0001,16,cuda
9,1,10,0.003182,0.988205,0.020379,0.885135,51.226197,525.158186,0.0001,16,cuda


Saved model and optimizer at 2,3.Sarcasm_Original.pth


# Prediction of male text and female text

In [30]:
test = ndata[ndata['test']==1]
class BertDataset(Dataset):
    def __init__(self, data, tokenizer, gender, max_length = 512):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,3]
        self.max_length=max_length
        self.gender = gender
    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):
        if self.gender=='male':
            text1 = self.train_csv.iloc[index,0] #0th coumn meanS male
        else:
            text1 = self.train_csv.iloc[index,1] #1th coumn meanS female  

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])


dataset_test_male= BertDataset(test, tokenizer, gender = 'male')
test_loader_male=DataLoader(dataset=dataset_test_male,batch_size=16, shuffle=False)
dataset_test_female = BertDataset(test, tokenizer, gender = 'female')
test_loader_female=DataLoader(dataset=dataset_test_female,batch_size=16, shuffle=False)

In [31]:
import warnings 
warnings.filterwarnings('ignore') 

p, emb_male = get_prediction(test_loader_male, ori_model)
test['pred_male_fod'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader_female, ori_model)
test['pred_female_fod'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_fod']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_fod']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))

100%|██████████| 116/116 [00:08<00:00, 13.63it/s]

For Male:
Total test data:  1850 Total mismatch:  218 Accuracy:  0.8821621621621621
For Female:
Total test data:  1850 Total mismatch:  221 Accuracy:  0.8805405405405405
Male Female Mismatch:  73


# FOD Result

In [33]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []
    
    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])
    
    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['pred_female_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))
test.to_csv('2,3.revision_fod_sarcasm_result.csv', index = False)

Male: 
Accuracy: 0.8822
95% Confidence Interval: (np.float64(0.8678243243243243), np.float64(0.8962162162162162))
mean accuracy: 0.8819989189189189

Female: 
Accuracy: 0.8805
95% Confidence Interval: (np.float64(0.8667432432432433), np.float64(0.8954189189189189))
mean accuracy: 0.8803664864864865
Male Female Mismatch:  73


In [2]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('2,3.revision_fod_sarcasm_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_fod.values
female_pred = test.pred_female_fod.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

SPD 95% Confidence Interval: [0.000, 0.011]	SPD Mean: 0.004
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.008]	EOD Mean: 0.003


44

,male,female,_original_label,test,gm_male,gm_female,Gender and Name Masked,pred_male_fod,pred_female_fod
3,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুত...,নিশ্চয় মহিলা সাংবাদিককে কোনো কিছু দেওয়ার প্রতি...,1,1,নিশ্চয় সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ্রুতি...,নিশ্চয় <Gender> সাংবাদিককে কোনো কিছু দেওয়ার প্...,নিশ্চয় <Gender> সাংবাদিককে কোনো কিছু দেওয়ার প্...,1,1
7,এই হারামজাদা গুলো প্রভাব খাটাতে পারে শুধুমাত্র...,এই হারামজাদী গুলো প্রভাব খাটাতে পারে শুধুমাত্র...,1,1,এই <Gender> গুলো প্রভাব খাটাতে পারে শুধুমাত্র ...,এই <Gender> গুলো প্রভাব খাটাতে পারে শুধুমাত্র ...,এই <Gender> গুলো প্রভাব খাটাতে পারে শুধুমাত্র ...,1,1
9,অনেকদিন দিন পর কাকু কে দেখলাম ।,অনেকদিন দিন পর কাকী কে দেখলাম ।,1,1,অনেকদিন দিন পর <Gender> কে দেখলাম ।,অনেকদিন দিন পর <Gender> কে দেখলাম ।,অনেকদিন দিন পর <Gender> কে দেখলাম ।,0,0
16,শিকড় থেকে শেখড়ে দুর্নীতি যখন সর্বত্র সেখানে ...,শিকড় থেকে শেখড়ে দুর্নীতি যখন সর্বত্র সেখানে ...,1,1,শিকড় থেকে শেখড়ে দুর্নীতি যখন সর্বত্র সেখানে ...,শিকড় থেকে শেখড়ে দুর্নীতি যখন সর্বত্র সেখানে ...,শিকড় থেকে শেখড়ে দুর্নীতি যখন সর্বত্র সেখানে ...,1,1
19,বাসায় থাকলে তো আমি জুতা ই খুঁজে পাইনা । মাঝে...,বাসায় থাকলে তো আমি জুতা ই খুঁজে পাইনা । মাঝে ম...,1,1,বাসায় থাকলে তো আমি জুতা ই খুঁজে পাইনা । মাঝে ম...,বাসায় থাকলে তো আমি জুতা ই খুঁজে পাইনা । মাঝে ম...,বাসায় থাকলে তো আমি জুতা ই খুঁজে পাইনা । মাঝে ম...,1,1
...,...,...,...,...,...,...,...,...,...
9224,বিশ্বের সব থেকে প্রতিভাবান শিশু জন্ম নিক এই দো...,বিশ্বের সব থেকে প্রতিভাবান শিশু জন্ম নিক এই দো...,0,1,বিশ্বের সব থেকে প্রতিভাবান শিশু জন্ম নিক এই দো...,বিশ্বের সব থেকে প্রতিভাবান শিশু জন্ম নিক এই দো...,বিশ্বের সব থেকে প্রতিভাবান শিশু জন্ম নিক এই দো...,0,0
9229,ভাই নিজে রিলেশন করে অভিজ্ঞতা থেকে আমাদের সতর্ক...,বোন নিজে রিলেশন করে অভিজ্ঞতা থেকে আমাদের সতর্ক...,0,1,<Gender> নিজে রিলেশন করে অভিজ্ঞতা থেকে আমাদের ...,<Gender> নিজে রিলেশন করে অভিজ্ঞতা থেকে আমাদের ...,<Gender> নিজে রিলেশন করে অভিজ্ঞতা থেকে আমাদের ...,0,0
9236,আরো বেশি আন্তরিক হতে রাতে ঘুরে ঘুরে অনাথ দু্ঃখ...,আরো বেশি আন্তরিক হতে রাতে ঘুরে ঘুরে অনাথিনী দু...,0,1,আরো বেশি আন্তরিক হতে রাতে ঘুরে ঘুরে <Gender> দ...,আরো বেশি আন্তরিক হতে রাতে ঘুরে ঘুরে <Gender> দ...,আরো বেশি আন্তরিক হতে রাতে ঘুরে ঘুরে <Gender> দ...,1,1
9238,কয়েকদিন পরে নিউজ আসবে ছাত্রলীগ কর্মী পড়াশোনার ...,কয়েকদিন পরে নিউজ আসবে ছাত্রলীগ মহিলা কর্মী পড়া...,0,1,কয়েকদিন পরে নিউজ আসবে ছাত্রলীগ <Gender> পড়াশোন...,কয়েকদিন পরে নিউজ আসবে ছাত্রলীগ <Gender> <Gende...,কয়েকদিন পরে নিউজ আসবে ছাত্রলীগ <Gender> <Gende...,1,1


Male: 
Accuracy: 0.9680
95% Confidence Interval: (0.9577319587628866, 0.9783505154639175)
mean accuracy: 0.9679773195876289

Female: 
Accuracy: 0.9515
95% Confidence Interval: (0.9381443298969072, 0.965979381443299)
mean accuracy: 0.9516701030927837

# Sentence Embedding Visualization for Original Finetuned Model

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(train, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb = get_prediction(train_loader, masked_model)

  0%|          | 0/233 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).

100%|██████████| 233/233 [00:51<00:00,  4.56it/s]


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb)

labels = train['original_lebel']   # Extract labels
review_texts = train['Gender and Name Masked']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Review Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Review Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings (Finetuned on Original Text) with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


# Prediction of NER Converted text data

In [ ]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])
dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, ori_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred_original'] != test['pred_ner']) )

100%|██████████| 14/14 [00:02<00:00,  4.98it/s]

Total test data:  218 Total mismatch:  54 Accuracy:  0.7522935779816513 Bias:  32


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred_original'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  54 Accuracy:  0.7522935779816513 Bias:  32


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred_original'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  54 Accuracy:  0.7522935779816513 Bias:  30


In [ ]:
y = test[test['pred']!=test['pred_original']][['Original Sentence', 'Gender and Name Masked', 'original_lebel', 'pred', 'pred_original']]
y

,Original Sentence,Gender and Name Masked,original_lebel,pred,pred_original
11,"মেয়েরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...",<Gender> টেকায় না পড়লে কখনো <Gender> কাছে জায় ...,0,0,1
709,"অনেক ইচ্ছে ছিল নাটক টা দেখব, শামীম ভাইকে দেখে,...",অনেক ইচ্ছে ছিল নাটক টা দেখব <Name> <Gender> দ...,0,1,0
711,"অনেক ইচ্ছে ছিল নাটক টা দেখব, শামীম ভাইকে দেখে,...",অনেক ইচ্ছে ছিল নাটক টা দেখব <Name> <Gender> দ...,0,1,0
818,এই নাটক সকালে ১০ টায় ঘুমিয়ে যে স্বপ্ন দেকে পোল...,এই নাটক সকালে ১০ টায় ঘুমিয়ে যে স্বপ্ন দেকে পোল...,0,0,1
978,বার বার নাটকের মাঝে এড দিলে ভাইয়া কোমন লাগে,বার বার নাটকের মাঝে এড দিলে <Gender> কোমন লাগে,0,1,0
1029,এরকম সুন্দর সুন্দর সামাজিক নাটক আমরা আরো চাই আ...,এরকম <Gender> <Gender> সামাজিক নাটক আমরা আরো চ...,1,0,1
3282,বড় ছেলে বিয়ে করল batch এ এসে। সব কিছু মিলিয়ে ...,বড় <Gender> বিয়ে করল batch এ এসে। সব কিছু মিলি...,1,0,1
3295,বর ছৈলৈ দৈখৈ আমার চৌখ দিয়ৈ পানি পরৈ nice,<Gender> ছৈলৈ দৈখৈ আমার চৌখ দিয়ৈ পানি পরৈ nice,1,0,1
3840,একটা ছেলের সাথে প্রেম করতে তার টাকা পয়সা কে ন...,একটা ছেলের সাথে প্রেম করতে তার টাকা পয়সা কে ন...,1,0,1
4192,"ছেলে দুটো জাতির ভাতিজা, কি সুন্দর চেহারা, কি স...","<Gender> দুটো জাতির ভাতিজা, কি <Gender> চেহারা...",1,0,1


In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/senti_reSult.csv', index = False)